In [ ]:
import os
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns

from contextlib import contextmanager
from time import time
from tqdm import tqdm
import lightgbm as lgbm
import category_encoders as ce

from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Target setting

In [ ]:
data0=pd.read_csv('../input/shark-attack-dataset/attacks.csv')
data0

In [ ]:
print(data0.columns.tolist())

In [ ]:
data1=data0.loc[0:6301]
data1=data1.drop(['Unnamed: 22', 'Unnamed: 23'],axis=1)
data2=data1[['Year', 'Type', 'Country', 'Area', 'Location', 'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time', 'Species ', ]]
data2[0:2].T

In [ ]:
data2=data2.fillna('NaN')
data2[0:2].T

In [ ]:
print(data2.columns.tolist())

In [ ]:
year=[]
for item in data2['Year']:
    if item=='NaN':
        year+=['NaN']
    elif item<1000:
        year+=['NaN']
    else:
        year+=[int(item)]
data2['Year']=year

In [ ]:
Name1=list(data2['Type'].unique())
print(Name1)
Name2=['Boating', 'Unprovoked', 'NaN', 'Provoked','NaN','Sea Disaster', 'NaN', 'Boating', 'Boating']
normal_mapping=dict(zip(Name1,Name2)) 

In [ ]:
data2['Type']=data2['Type'].map(normal_mapping)

In [ ]:
Name1a=list(data2['Fatal (Y/N)'].unique())
print(Name1a)
Name2a=['N', 'Y', 'NaN', 'NaN', 'NaN', 'NaN', 'N', 'N', 'Y']
normal_mapping_a=dict(zip(Name1a,Name2a)) 

In [ ]:
data2['Fatal (Y/N)']=data2['Fatal (Y/N)'].map(normal_mapping_a)

In [ ]:
drop_index=data2[data2['Fatal (Y/N)']=='NaN'].index.tolist()
print(len(drop_index))
print(drop_index)
data2=data2.drop(index=data2.index[drop_index]).reset_index(drop=True)
data2

In [ ]:
list_cols = ['Year', 'Type', 'Country', 'Area', 'Location', 'Activity', 'Name', 'Sex ', 
             'Age', 'Injury', 'Fatal (Y/N)', 'Time', 'Species ']
ce_ohe = ce.OrdinalEncoder(cols=list_cols,handle_unknown='impute')
data3 = ce_ohe.fit_transform(data2)
data3

In [ ]:
target=['Fatal (Y/N)']
datay=data3['Fatal (Y/N)']
datax=data3.drop('Fatal (Y/N)',axis=1)

In [ ]:
print(datay[0:2].T)
print()
print(datax[0:2].T)

In [ ]:
n=len(data3)
print(n)

In [ ]:
N=[]
for i in range(n):
    N+=[i]
random.shuffle(N)

In [ ]:
trainX=datax.loc[N[0:(n//4)*3]]
trainY=datay.loc[N[0:(n//4)*3]]
testX=datax.loc[N[(n//4)*3:]]
testY=datay.loc[N[(n//4)*3:]]

In [ ]:
print(trainX.shape)
print(testX.shape)

In [ ]:
df_columns = list(datax.columns)
print(df_columns)

In [ ]:
train_df=trainX
test_df=testX

In [ ]:
train_df.columns=df_columns
test_df.columns=df_columns

In [ ]:
def create_numeric_feature(input_df):
    use_columns = df_columns 
    return input_df[use_columns].copy()

In [ ]:
from contextlib import contextmanager
from time import time

class Timer:
    def __init__(self, logger=None, format_str='{:.3f}[s]', prefix=None, suffix=None, sep=' '):

        if prefix: format_str = str(prefix) + sep + format_str
        if suffix: format_str = format_str + sep + str(suffix)
        self.format_str = format_str
        self.logger = logger
        self.start = None
        self.end = None

    @property
    def duration(self):
        if self.end is None:
            return 0
        return self.end - self.start

    def __enter__(self):
        self.start = time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time()
        out_str = self.format_str.format(self.duration)
        if self.logger:
            self.logger.info(out_str)
        else:
            print(out_str)

In [ ]:
from tqdm import tqdm

def to_feature(input_df):

    processors = [
        create_numeric_feature,
    ]
    
    out_df = pd.DataFrame()
    
    for func in tqdm(processors, total=len(processors)):
        with Timer(prefix='create' + func.__name__ + ' '):
            _df = func(input_df)

        assert len(_df) == len(input_df), func.__name__
        out_df = pd.concat([out_df, _df], axis=1)
        
    return out_df

In [ ]:
train_feat_df = to_feature(train_df)
test_feat_df = to_feature(test_df)

# Model

In [ ]:
import lightgbm as lgbm
from sklearn.metrics import mean_squared_error

def fit_lgbm(X, y, cv, 
             params: dict=None, 
             verbose: int=50):

    if params is None:
        params = {}

    models = []
    oof_pred = np.zeros_like(y, dtype=np.float)

    for i, (idx_train, idx_valid) in enumerate(cv): 
        x_train, y_train = X[idx_train], y[idx_train]
        x_valid, y_valid = X[idx_valid], y[idx_valid]

        clf = lgbm.LGBMRegressor(**params)
        
        with Timer(prefix='fit fold={} '.format(i)):
            clf.fit(x_train, y_train, 
                    eval_set=[(x_valid, y_valid)],  
                    early_stopping_rounds=100,
                    verbose=verbose)

        pred_i = clf.predict(x_valid)
        oof_pred[idx_valid] = pred_i
        models.append(clf)
        print(f'Fold {i} RMSLE: {mean_squared_error(y_valid, pred_i) ** .5:.4f}')
        print()

    score = mean_squared_error(y, oof_pred) ** .5
    print('-' * 50)
    print('FINISHED | Whole RMSLE: {:.4f}'.format(score))
    return oof_pred, models

In [ ]:
params = {
    'objective': 'rmse', 
    'learning_rate': .1,
    'reg_lambda': 1.,
    'reg_alpha': .1,
    'max_depth': 5, 
    'n_estimators': 10000, 
    'colsample_bytree': .5, 
    'min_child_samples': 10,
    'subsample_freq': 3,
    'subsample': .9,
    'importance_type': 'gain', 
    'random_state': 71,
    'num_leaves': 62
}

In [ ]:
y = trainY
print(y.shape)
ydf=pd.DataFrame(y)
ydf

In [ ]:
from sklearn.model_selection import KFold

for i in range(1):
    fold = KFold(n_splits=5, shuffle=True, random_state=71)
    ydfi=ydf.iloc[:,i]
    y=np.array(ydfi)
    cv = list(fold.split(train_feat_df, y))
    oof, models = fit_lgbm(train_feat_df.values, y, cv, params=params, verbose=500)
    
    fig,ax = plt.subplots(figsize=(6,6))
    ax.set_title(target[i],fontsize=20)
    ax.set_xlabel('trainY of '+target[i],fontsize=12)
    ax.set_ylabel('oof of '+target[i],fontsize=12)
    ax.scatter(y,oof)

In [ ]:
print(target)

# Visualize Importance and Predict

In [ ]:
def visualize_importance(models, feat_train_df):

    feature_importance_df = pd.DataFrame()
    for i, model in enumerate(models):
        _df = pd.DataFrame()
        _df['feature_importance'] = model.feature_importances_
        _df['column'] = feat_train_df.columns
        _df['fold'] = i + 1
        feature_importance_df = pd.concat([feature_importance_df, _df], 
                                          axis=0, ignore_index=True)

    order = feature_importance_df.groupby('column')\
        .sum()[['feature_importance']]\
        .sort_values('feature_importance', ascending=False).index[:50]

    fig, ax = plt.subplots(figsize=(8, max(6, len(order) * .25)))
    sns.boxenplot(data=feature_importance_df, 
                  x='feature_importance', 
                  y='column', 
                  order=order, 
                  ax=ax, 
                  palette='viridis', 
                  orient='h')
    
    ax.tick_params(axis='x', rotation=0)
    #ax.set_title('Importance')
    ax.grid()
    fig.tight_layout()
    
    return fig,ax

#fig, ax = visualize_importance(models, train_feat_df)

In [ ]:
pred1=[]
for i in range(1):
    fold = KFold(n_splits=5, shuffle=True, random_state=71)
    ydfi=ydf.iloc[:,i]
    y=np.array(ydfi)
    cv = list(fold.split(train_feat_df, y))
    oof, models = fit_lgbm(train_feat_df.values, y, cv, params=params, verbose=500)
    fig, ax = visualize_importance(models, train_feat_df)
    ax.set_title(target[i]+' Imortance',fontsize=20)
    pred0 = np.array([model.predict(test_feat_df.values) for model in models])
    pred1+=[np.mean(pred0, axis=0)]

## For fatality('Fatal (Y/N)'), 'Injury' is the most important factor.

In [ ]:
data2[data2['Fatal (Y/N)']=='Y']['Injury'].unique()[0:100]

In [ ]:
df = data2[data2['Fatal (Y/N)']=='Y']['Injury']

In [ ]:
import re
import nltk
nltk.download("stopwords")
nltk.download('punkt')
from nltk import word_tokenize,sent_tokenize
nltk.download('wordnet')
import nltk as nlp

In [ ]:
cult_list=[]

for cult in df:
    cult=re.sub("[^a-zA-z]"," ",cult)
    cult=cult.lower()
    cult=nltk.word_tokenize(cult)
    lemma=nlp.WordNetLemmatizer()
    cult=[lemma.lemmatize(word) for word in cult]
    cult=" ".join(cult)
    cult_list.append(cult)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
max_features=200
count_vectorizer=CountVectorizer(max_features=max_features,stop_words="english")
sparce_matrix=count_vectorizer.fit_transform(cult_list).toarray()

In [ ]:
data3 = pd.DataFrame(count_vectorizer.get_feature_names(),columns=["Words"])
data3

In [ ]:
random.seed(2019)
random.shuffle(data3.Words)
data3[0:10]

In [ ]:
from wordcloud import WordCloud 
import matplotlib.pyplot as plt

plt.subplots(figsize=(16,16))
wordcloud=WordCloud(background_color="black",width=1024,height=768).generate(" ".join(data3.Words))
plt.imshow(wordcloud)
plt.axis("off")
plt.show()